In [1]:
%store -r __importRegression

In [2]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from itertools import product
from sklearn.linear_model import LinearRegression
import gc

In [4]:
#sales = pd.read_csv('../input/train.csv')
#all_data = pd.read_csv('all_data.csv')
all_data = pd.read_csv('../input/all_data_1_2_3_4_5_12_cat.csv')
test = pd.read_csv('../input/test_1_2_3_4_5_12_cat.csv')
#test = pd.read_csv('../input/test.csv')
#items = pd.read_csv("../input/items.csv")
#categories = pd.read_csv("../input/item_categories.csv")
#print (sales.shape)

In [5]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [6]:
# Add date_block_num for test with value 34 (next month)
test['date_block_num'] = 34

In [7]:
all_data, NAs = reduce_mem_usage(all_data)

('Memory usage of properties dataframe is :', 2081, ' MB')
******************************
('Column: ', 'shop_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'item_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint16'))
******************************
******************************
('Column: ', 'date_block_num')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'target')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
******************************
******************************
('Column: ', 'target_shop')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
******************************
******************************
('Column: ', 'target_item')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
***************************

In [8]:
test, NAs = reduce_mem_usage(test)

('Memory usage of properties dataframe is :', 37, ' MB')
******************************
('Column: ', 'ID')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint32'))
******************************
******************************
('Column: ', 'shop_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'item_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint16'))
******************************
******************************
('Column: ', 'date_block_num')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'target_lag_1')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
******************************
******************************
('Column: ', 'target_item_lag_1')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint16'))
************************

In [9]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [10]:
all_data.head()

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,...,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5
0,0,19,0,0,5578,1,0,0,0,0,...,0,0,0,40,0,0,0,0,0,0
1,0,27,0,0,5578,7,0,0,0,0,...,0,0,0,19,0,0,0,0,0,0
2,0,28,0,0,5578,8,0,0,0,0,...,0,0,0,30,0,0,0,0,0,0
3,0,29,0,0,5578,4,0,0,0,0,...,0,0,0,23,0,0,0,0,0,0
4,0,32,0,6,5578,299,0,0,0,0,...,0,0,0,40,0,0,0,0,0,0


In [ ]:
all_data.target.plot()

In [ ]:
all_data.target.describe()

In [ ]:

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data, NAS = reduce_mem_usage(all_data)

del grid, gb 
gc.collect();

In [ ]:
all_data.head()

In [ ]:
all_data.to_csv('all_data.csv', index=False)

In [ ]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols))

In [ ]:
cols_to_rename

In [ ]:
cols_to_rename = ['target', 'target_item', 'target_shop']

In [11]:
# lag months: 1 = last month sales; 12 = last year same month sales
shift_range = [1, 2, 3, 4, 5, 12]

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)
    
    # Merge lag feature with train data
    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
    
    # Merge lag feature with test data
    test = pd.merge(test, train_shift, on=index_cols, how='left').fillna(0)
    
del train_shift
gc.collect();

In [ ]:
items = pd.read_csv("../input/items.csv")
#items, NAs = reduce_mem_usage(items)
print items.shape

In [ ]:
items.head()

In [ ]:
# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')

test = pd.merge(test, item_category_mapping, how='left', on='item_id')

In [12]:
test.head()

,ID,shop_id,item_id,date_block_num,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,...,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5
0,0,5,5037,34,0,25,1052,1,110,1092,...,1,65,1445,19,1,54,991,1,105,954
1,1,5,5320,34,0,0,0,0,0,0,...,0,0,0,55,0,0,0,0,0,0
2,2,5,5233,34,1,42,1052,3,80,1092,...,0,0,0,19,0,37,991,2,119,954
3,3,5,5232,34,0,28,1052,0,48,1092,...,0,0,0,23,0,0,0,0,0,0
4,4,5,5268,34,0,0,0,0,0,0,...,0,0,0,20,0,0,0,0,0,0


In [ ]:
test, NAs = reduce_mem_usage(test)

In [ ]:
test.to_csv('../input/test_1_2_3_4_5_12_cat.csv', index=False)

In [ ]:
all_data, NAs = reduce_mem_usage(all_data)

In [ ]:
all_data.to_csv('../input/all_data_1_2_3_4_5_12_cat.csv', index=False)

In [13]:
## Add lag feature differences (feature interaction)
for i in [2,3,4,5]:
    fname = '_lag' + str(i-1) + '_lag' + str(i)
    lag1 = '_lag_' + str(i-1)
    lag2 = '_lag_' + str(i)
    all_data['target' + fname] = all_data['target' + lag1] - all_data['target' + lag2]
    test['target' + fname] = test['target' + lag1] - test['target' + lag2]
    all_data['target_item' + fname] = all_data['target_item' + lag1] - all_data['target_item' + lag2]
    test['target_item' + fname] = test['target_item' + lag1] - test['target_item' + lag2]
    all_data['target_shop' + fname] = all_data['target_shop' + lag1] - all_data['target_shop' + lag2]
    test['target_shop' + fname] = test['target_shop' + lag1] - test['target_shop' + lag2]

In [14]:
print (all_data.shape, test.shape)

((10913850, 37), (214200, 35))


In [15]:
# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
fit_cols.append('item_category_id')
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

print fit_cols
print to_drop_cols

['target_lag_1', 'target_item_lag_1', 'target_shop_lag_1', 'target_lag_2', 'target_item_lag_2', 'target_shop_lag_2', 'target_lag_3', 'target_item_lag_3', 'target_shop_lag_3', 'target_lag_12', 'target_item_lag_12', 'target_shop_lag_12', 'target_lag_4', 'target_item_lag_4', 'target_shop_lag_4', 'target_lag_5', 'target_item_lag_5', 'target_shop_lag_5', 'target_lag1_lag2', 'target_item_lag1_lag2', 'target_shop_lag1_lag2', 'target_lag2_lag3', 'target_item_lag2_lag3', 'target_shop_lag2_lag3', 'target_lag3_lag4', 'target_item_lag3_lag4', 'target_shop_lag3_lag4', 'target_lag4_lag5', 'target_item_lag4_lag5', 'target_shop_lag4_lag5', 'item_category_id']
['target_item', 'target_shop', 'target', 'date_block_num']


In [16]:
# Drop data from 2013
all_data = all_data[all_data.date_block_num >= 12]

In [ ]:
all_data, NAs = reduce_mem_usage(all_data)

## SVD

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = all_data.drop(to_drop_cols, axis=1)

In [ ]:
X.head()

In [ ]:
## Standardize the data
X_std = StandardScaler().fit_transform(X)

In [ ]:
## Get Covariance matrix

cov_mat = np.cov(X_std.T)
print('Covariance matrix shape', cov_mat.shape)

In [ ]:
## Getting eigen vectors and eigen values
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [ ]:
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
## Making sure eigen vectors have a norm of 1
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

In [ ]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(10, 6))

    plt.bar(range(33), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(33), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()

We can see that the first 25 eigenvalues explains most of the variance. Now that we now that we want the top 25 eigenvectors, we can use the PCA function from sklearn

In [ ]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(33,1),
                      eig_pairs[1][1].reshape(33,1)))

In [ ]:
X_2 = X_std.dot(matrix_w)

In [ ]:
X_2.shape

In [ ]:
all_data.shape

In [ ]:
X_2 = pd.DataFrame(X_2, columns=['PCA_1', 'PCA_2'])

In [ ]:
X_2.head()

In [ ]:
all_data['PCA_1'] = X_2['PCA_1']
all_data['PCA_2'] = X_2['PCA_2']

In [ ]:
all_data.head()

## Train / Validation split

In [17]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [ ]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'n_jobs': -1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = LGBMRegressor(**lgb_params)

In [18]:
model = LGBMRegressor(n_jobs=-1, metric='rmse', objective='mse')

In [ ]:
model = RandomForestRegressor(n_jobs=-1)

In [ ]:
model = LinearRegression()

## Target values clipping

In [19]:
all_data.target.describe()

count    6.425094e+06
mean     3.245825e-01
std      3.615032e+00
min     -5.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.253000e+03
Name: target, dtype: float64

In [20]:
# Clipping to 40 seems to improve RMSE score.
# Tune this value to further improve score?
all_data['target'] = all_data.target.clip(lower=-1, upper=40)

In [21]:
all_data.target.describe()

count    6.425094e+06
mean     2.992826e-01
std      1.435029e+00
min     -1.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.000000e+01
Name: target, dtype: float64

In [ ]:
# Moving window validation scheme.
# On each iteration, use last month for validation
validation_months = [33, 32, 31, 30, 29]

for last_month in validation_months:
    # Split train and validation data
    dates_train = dates[dates <  last_month]
    dates_test  = dates[dates == last_month]

    X_train = all_data.loc[dates <  last_month].drop(to_drop_cols, axis=1)
    X_test =  all_data.loc[dates == last_month].drop(to_drop_cols, axis=1)

    y_train = all_data.loc[dates <  last_month, 'target'].values
    y_test =  all_data.loc[dates == last_month, 'target'].values
    
    model.fit(X_train, y_train)

    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    ## R2 and RMSE score for each validation fold
    print('Month {0:d} Test R-2: {1:f}'.format(last_month, r2_score(y_test, pred_test)))
    print('Month {0:d} Test RMSE {1:f}'.format(last_month, np.sqrt(mean_squared_error(y_test, pred_test))))
    
del dates_train, dates_test, X_train, X_test, y_train, y_test, pred_train, pred_test
gc.collect()

## Prepare submission

In [22]:
## Take all train data
X_train_all = all_data.drop(to_drop_cols, axis=1)
y_train_all = all_data.target.values

In [ ]:
y = pd.DataFrame(y_train_all)

In [ ]:
y.describe()

In [ ]:
X_train_all.head()

In [ ]:
X_train_all.target_lag_1.describe()

In [ ]:
test.sort(columns='ID')

In [23]:
test_id = test.pop('ID')
test.drop('date_block_num', axis=1, inplace=True)

In [24]:
test.head()

,shop_id,item_id,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,...,target_shop_lag1_lag2,target_lag2_lag3,target_item_lag2_lag3,target_shop_lag2_lag3,target_lag3_lag4,target_item_lag3_lag4,target_shop_lag3_lag4,target_lag4_lag5,target_item_lag4_lag5,target_shop_lag4_lag5
0,5,5037,0,25,1052,1,110,1092,3,119,...,65496,-2,-9,65334,2,65,303,0,-51,37
1,5,5320,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,5233,1,42,1052,3,80,1092,1,150,...,65496,2,-70,65334,1,113,303,-2,-82,37
3,5,5232,0,28,1052,0,48,1092,1,65,...,65496,-1,-17,65334,1,65,1294,0,0,0
4,5,5268,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_train_all.head()

,shop_id,item_id,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,...,target_shop_lag1_lag2,target_lag2_lag3,target_item_lag2_lag3,target_shop_lag2_lag3,target_lag3_lag4,target_item_lag3_lag4,target_shop_lag3_lag4,target_lag4_lag5,target_item_lag4_lag5,target_shop_lag4_lag5
4488756,2,27,0,4,1322,0,2,862,0,6,...,460,0,-4,67,0,5,65386,0,0,70
4488757,2,30,0,47,1322,0,46,862,0,24,...,460,0,22,67,0,11,65386,0,-6,70
4488758,2,31,0,25,1322,0,27,862,0,25,...,460,0,2,67,0,6,65386,0,-6,70
4488759,2,32,0,89,1322,0,81,862,0,58,...,460,0,23,67,0,-1,65386,0,-13,70
4488760,2,33,1,42,1322,2,15,862,0,33,...,460,2,-18,67,0,-10,65386,0,8,70


In [26]:
model.fit(X_train_all, y_train_all)

pred_train_all = model.predict(X_train_all)
pred_test = model.predict(test)

## R2 and RMSE score for each validation fold
print('Train R-2: {0:f}'.format(r2_score(y_train_all, pred_train_all)))
print('Train RMSE {0:f}'.format(np.sqrt(mean_squared_error(y_train_all, pred_train_all))))

Train R-2: 0.381726
Train RMSE 1.128369


In [ ]:
pred_test.shape

In [27]:
for i in range(len(pred_test)):
    if pred_test[i] > 20:
        pred_test[i] = 20
    if pred_test[i] < 0:
        pred_test[i] = 20

In [28]:
test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': pred_test})
print test_submit.shape
test_submit.to_csv('new_lgb_Clip40_before_train.csv', index=False)
test_submit.head()

(214200, 2)


,ID,item_cnt_month
0,0,0.407095
1,1,0.231380
2,2,0.647920
3,3,0.324343
4,4,2.897320
